## Apa itu ARIMA?

ARIMA, singkatan dari 'Auto Regressive Integrated Moving Average' sebenarnya adalah kelas model yang 'menjelaskan' deret waktu tertentu berdasarkan nilai sebelumnya, yaitu lag-nya sendiri dan kesalahan perkiraan lag, sehingga persamaan dapat digunakan untuk memprediksi nilai masa depan.




## Persiapan Dataset

In [ ]:
! wget https://www.dropbox.com/s/9bfx2ojb6l4uqen/www-usage.csv

In [ ]:
! head www-usage.csv

In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot  as  plt
from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
# df = pd.read_csv("www-usage.csv", header=0, index_col=0)
df = pd.read_csv("www-usage.csv", header=0)

In [ ]:
df.head(3)

In [ ]:
df.describe()

In [ ]:
df.plot(y=["usage"], figsize = (20,6))

In [ ]:
df["rolling_avg"] = df["usage"].rolling(window=12).mean()
df.plot(y=["usage", "rolling_avg"], figsize = (20,6))

## Preprocessing

In [ ]:
df_clean = df[["time", "usage", "rolling_avg"]].copy()

In [ ]:
df_clean.head(5)

## Menentukan Order Differencing,

Istilah 'Auto Regressive' dalam ARIMA adalah model regresi linier yang menggunakan lagnya sendiri (nilai sebelumnya) sebagai prediktor atau independent variable. Model regresi linier bekerja dengan sangat baik ketika prediktor tidak berkorelasi dan independen satu sama lain. Data deret waktu harus dibuat stasioner untuk menghilangkan korelasi dan kolinearitas yang jelas dengan data masa lalu.


Plot grafik autokorelasi dataset menggunakan plot_acf

In [ ]:
f = plt.figure(figsize=(20, 6))
ax1 = f.add_subplot(121)
ax1.set_title('Zero Order Differencing')
ax1.plot(df_clean['usage'])

ax2 = f.add_subplot(122)
plot_acf(df_clean['usage'], ax = ax2)

plt.show()

Dari grafik autokorelasi terlihat bahwa setiap lag masih memiliki nilai yang significant. Artinya dataset masih belum stasioner.


### Bagaimana membuat dataset menjadi stasioner?

Pendekatan yang paling umum digunakan adalah dengan difference, yaitu mengurangi nilai sebelumnya dengan nilai saat ini. Nilai **d** adalah jumlah diferensial minimum yang diperlukan untuk membuat deret tersebut stasioner. 

In [ ]:
f = plt.figure(figsize=(20, 12))
ax1 = f.add_subplot(221)
ax1.set_title('1st Order Differencing')
ax1.plot(df_clean['usage'].diff())

ax2 = f.add_subplot(222)
plot_acf(df_clean['usage'].diff().dropna(), ax = ax2)

ax3 = f.add_subplot(223)
ax3.set_title('2nd Order Differencing')
ax3.plot(df_clean['usage'].diff().diff())

ax4 = f.add_subplot(224)
plot_acf(df_clean['usage'].diff().diff().dropna(), ax = ax4)

plt.show()

Pada 2nd Order Differencing, lag 1 sudah tidak memiliki autokorelasi yang significant. Dapat kita simpulkan nilai d adalah 2

### Augmented Dickey-Fuller

Cara lain untuk menentukan nilai order differencing adalah menggunakan algoritma Augmented Dickey-Fuller. Algoritma ini adalah perhitungan matematis untuk menentukan p-value. Untuk menentukan order differencing, adalah dengan membandingkan dengan nilai ambang batas atau tingkat signifikansi 0,05. Jika p-value lebih kecil dari level ini, maka data tersebut stasioner

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(df_clean['usage'].dropna())
print('p-value order 0: ', result[1])

result = adfuller(df_clean['usage'].diff().dropna())
print('p-value order 1: ', result[1])

result = adfuller(df_clean['usage'].diff().diff().dropna())
print('p-value order 2: ', result[1])

Seperti yang kita lihat di atas, setelah **order differencing ke-2, nilai p turun melampaui ambang batas yang dapat diterima**. Dengan demikian, kita dapat menganggap order differencing (“d”) sebagai 2. Ini sesuai dengan grafik autokorelasi sebelumnya.

## Menentukan order ‘Auto Regressive’ (AR)

Model Auto Regressive (hanya AR) murni adalah model di mana Yt hanya bergantung pada lag-nya sendiri. Artinya, Yt adalah fungsi dari 'lag Yt'.

'p' adalah order dari 'Auto Regressive' (AR), yang mengacu pada jumlah lag dari Y yang akan digunakan sebagai prediktor. Nilai p dapat ditentukan dengan menggunakan grafik Partial Autocorrelation (PACF).

Partial Autocorrelation menggambarkan korelasi antara deret dan lag-nya, setelah membuang kontribusi dari intermediate lag. Jadi, PACF semacam menyampaikan korelasi murni antara lag dan seri.

Setiap autokorelasi dalam deret stasioner dapat diperbaiki dengan menambahkan menambahkan nilai p yang cukup. Jadi, kita awalnya mengambil urutan istilah AR sama dengan banyak lag yang melintasi batas signifikansi di plot PACF.

In [ ]:
f = plt.figure(figsize=(20, 12))
ax1 = f.add_subplot(221)
ax1.set_title('1st Order Differencing')
ax1.plot(df_clean['usage'].diff())

ax2 = f.add_subplot(222)
plot_pacf(df_clean['usage'].diff().dropna(), ax = ax2)

ax3 = f.add_subplot(223)
ax3.set_title('2nd Order Differencing')
ax3.plot(df_clean['usage'].diff().diff())

ax4 = f.add_subplot(224)
plot_pacf(df_clean['usage'].diff().diff().dropna(), ax = ax4)

plt.show()

Jika menggunakan d = 2, terlihat pada grafik PACF, lag 1 memiliki nilai tertinggi.


## Menentukan order ‘Moving Average’ (MA)

“q” dapat diperkirakan dengan cara yang sama dengan melihat plot ACF dan bukan plot PACF. Melihat jumlah lag yang melewati ambang batas, kita dapat menentukan seberapa banyak masa lalu yang cukup signifikan untuk dipertimbangkan di masa depan.

In [ ]:
f = plt.figure(figsize=(20, 6))
ax1 = f.add_subplot(121)
ax1.set_title('2nd Order Differencing')
ax1.plot(df_clean['usage'].diff().diff())

ax2 = f.add_subplot(122)
plot_pacf(df_clean['usage'].diff().diff().dropna(), ax = ax2)

plt.show()

Dari analisa diatas kita akan membuat model ARIMA(p=1, d=2,q= 1).

## Membuat Model ARIMA

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

model = ARIMA(df_clean['usage'], order=(1,2,1))
model_arima = model.fit()

In [ ]:
model_arima.plot_predict(dynamic = False)
plt.show()

In [ ]:
print(model_arima.summary())

Pada summary diatas terlihat:


*   nilai p untuk ar cukup besar, yaitu 0.147
*   nilai p untuk ma adalah 0

Nilai ideal untuk kedua nilai tersebut adalah < 0.05



### Perbaikan Model ARIMA (1,1,1)

In [ ]:
model = ARIMA(df_clean['usage'], order=(1,1,1))
model_arima = model.fit()

In [ ]:
print(model_arima.summary())

Terlihat hasil diatas, nilai p untuk ar dan ma adalah 0 atau < 0.05

AIC juga menurun dari 525.586 ke 515.579

Akaike Information Critera atau AIC merupakan salah satu parameter yang baik untuk menguji sebuah model secara matematis. Skor Kriteria Informasi mengukur jumlah informasi yang hilang melalui pelatihan dan generalisasi model ARIMA. Nilai AIC harus serendah mungkin.

ini berarti ARIMA (1,1,1) lebih baik dari model sebelumnya

In [ ]:
model_arima.plot_predict(dynamic = False)
plt.show()

In [ ]:
residuals = pd.DataFrame(model_arima.resid)

fig, ax = plt.subplots(1,2, figsize=(20,6))
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()